In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

In [3]:
repo_url = "https://github.com/aakashvardhan/s22-unet-vae.git"

local_dir = '/content/s22-unet-vae'



# Check if the local directory already exists
if not os.path.exists(local_dir):
    # Clone the repository because it does not exist
    !git clone {repo_url}
else:
    # Change directory to the local repository
    %cd {local_dir}
    # Pull the latest changes because the repository already exists
    !git pull

Cloning into 's22-unet-vae'...
remote: Enumerating objects: 179, done.
remote: Counting objects: 100% (179/179), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 179 (delta 90), reused 148 (delta 59), pack-reused 0
Receiving objects: 100% (179/179), 144.60 KiB | 800.00 KiB/s, done.
Resolving deltas: 100% (90/90), done.


In [4]:
%cd s22-unet-vae

/content/s22-unet-vae


In [5]:
import sys

sys.path.append('.')
sys.path.append('./models')
sys.path.append('./data')
sys.path.append('./notebooks')
sys.path.append('./utils')

In [6]:
!pip install -q -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.7/206.7 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.6/303.6 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.5/808.5 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.3/812.3 kB 56.3 MB/s eta 0:0

MP+Tr+BCE

In [7]:
%run trainer.py --config "training_1.json" --batch_size 64 --epochs 25

INFO: Seed set to 42
INFO:lightning.fabric.utilities.seed:Seed set to 42


Setting up DataModule...


100%|██████████| 791918971/791918971 [00:31<00:00, 25060430.53it/s]


Extracting data/oxford-iiit-pet/images.tar.gz to data/oxford-iiit-pet


100%|██████████| 19173078/19173078 [00:01<00:00, 10776559.84it/s]


...

Validation: |          | 0/? [00:00<?, ?it/s]


 Epoch: 25 | Val Loss: 0.91534 | Val Acc: 0.73253


Sample output images are saved at asset


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │    0.7325285077095032     │
│      val_loss_epoch       │    0.9153381586074829     │
└───────────────────────────┴───────────────────────────┘

Model evaluation complete.


MP+Tr+Dice Loss

In [8]:
%run trainer.py --config "training_2.json" --batch_size 64 --epochs 25

INFO: Seed set to 42
INFO:lightning.fabric.utilities.seed:Seed set to 42


Setting up DataModule...
DataModule setup complete.
Setting up UNet model...


epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇██
lr-Adam,▂▃▄▅▇████▇▇▆▆▅▄▄▃▃▂▂▁▁▁
lr-Adam-momentum,█▇▅▄▂▁▁▁▁▂▂▃▃▄▅▅▆▆▇▇███
train_acc,▁▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇████████
train_loss_epoch,█▆▅▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
train_loss_step,█▇▅▅▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁
trainer/global_step,▁▂▁▁▂▂▁▁▁▃▃▁▁▄▄▄▁▁▄▅▁▁▅▆▁▁▁▆▇▁▁▁▇█▂▂█▁▁█
val_acc,▂▃▁▆▅▅▅██
val_loss_epoch,▄▅▇▁▄▅█▄▄
val_loss_step,▄▃▃▄▃▆▄▅▅▆▄▄▅▅▁▁▂▂▅▂▂▄▅▃▄▄▂▅▅█▆▅▃▃▄▅▃▃▄▁
epoch,25


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO: Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:lightning.pytorch.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores

Finding the best learning rate...


...

Validation: |          | 0/? [00:00<?, ?it/s]


 Epoch: 25 | Val Loss: 0.36345 | Val Acc: 0.76100


Sample output images are saved at asset


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │    0.7609968781471252     │
│      val_loss_epoch       │    0.3634498417377472     │
└───────────────────────────┴───────────────────────────┘

Model evaluation complete.


StrConv+Tr+BCE

In [9]:
%run trainer.py --config "training_3.json" --batch_size 64 --epochs 25

INFO: Seed set to 42
INFO:lightning.fabric.utilities.seed:Seed set to 42


Setting up DataModule...
DataModule setup complete.
Setting up UNet model...


epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇██
lr-Adam,▂▃▄▅▇████▇▇▆▆▅▄▄▃▃▂▂▁▁▁
lr-Adam-momentum,█▇▅▄▂▁▁▁▁▂▂▃▃▄▅▅▆▆▇▇███
train_acc,▁▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇████████
train_loss_epoch,█▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
train_loss_step,█▇▅▅▅▅▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁
trainer/global_step,▁▂▁▁▂▂▁▁▁▃▃▁▁▄▄▄▁▁▄▅▁▁▅▆▁▁▁▆▇▁▁▁▇█▂▂█▁▁█
val_acc,▁▅▅▆▆▇▇██
val_loss_epoch,█▃▃▂▂▂▂▁▁
val_loss_step,██▇██▄▃▄▄▄▃▃▄▃▃▃▄▃▃▃▃▃▃▃▂▃▂▂▂▃▃▂▂▂▂▃▂▂▂▁
epoch,25


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO: Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:lightning.pytorch.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores

Finding the best learning rate...


...

Validation: |          | 0/? [00:00<?, ?it/s]


 Epoch: 25 | Val Loss: 0.29613 | Val Acc: 0.88623


Sample output images are saved at asset


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │    0.8862345218658447     │
│      val_loss_epoch       │    0.2961284816265106     │
└───────────────────────────┴───────────────────────────┘

Model evaluation complete.


StrConv+Ups+Dice Loss

In [10]:
%run trainer.py --config "training_4.json" --batch_size 64 --epochs 25

INFO: Seed set to 42
INFO:lightning.fabric.utilities.seed:Seed set to 42


Setting up DataModule...
DataModule setup complete.
Setting up UNet model...


epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇██
lr-Adam,▂▃▄▅▇████▇▇▆▆▅▄▄▃▃▂▂▁▁▁
lr-Adam-momentum,█▇▅▄▂▁▁▁▁▂▂▃▃▄▅▅▆▆▇▇███
train_acc,▁▄▄▅▆▆▆▆▆▇▇▇▇▇▇▇▇████████
train_loss_epoch,█▆▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
train_loss_step,█▆▅▅▄▄▃▄▃▂▂▃▂▂▂▁▂▂▂▁▂▁▁
trainer/global_step,▁▂▁▁▂▂▁▁▁▃▃▁▁▄▄▄▁▁▄▅▁▁▅▆▁▁▁▆▇▁▁▁▇█▂▂█▁▁█
val_acc,▂▁▅▆▆████
val_loss_epoch,██▅▃▃▁▁▁▁
val_loss_step,█▇█▇▇▆▆▇▇▅▄▄▄▅▃▃▃▃▃▃▃▃▂▂▂▂▁▁▁▂▂▂▂▁▂▂▁▂▁▁
epoch,25


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO: Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:lightning.pytorch.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores

Finding the best learning rate...


...

Validation: |          | 0/? [00:00<?, ?it/s]


 Epoch: 25 | Val Loss: 0.14531 | Val Acc: 0.89400


Sample output images are saved at asset


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │    0.8939970135688782     │
│      val_loss_epoch       │    0.14531075954437256    │
└───────────────────────────┴───────────────────────────┘

Model evaluation complete.


In [11]:
import wandb
wandb.finish()

epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇██
lr-Adam,▂▃▄▅▇████▇▇▆▆▅▄▄▃▃▂▂▁▁▁
lr-Adam-momentum,█▇▅▄▂▁▁▁▁▂▂▃▃▄▅▅▆▆▇▇███
train_acc,▁▄▅▆▆▆▆▇▇▇▇▇▇▇▇██████████
train_loss_epoch,█▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
train_loss_step,█▆▅▅▄▅▄▃▄▂▃▂▂▂▂▂▁▂▁▁▁▁▁
trainer/global_step,▁▂▁▁▂▂▁▁▁▃▃▁▁▄▄▄▁▁▄▅▁▁▅▆▁▁▁▆▇▁▁▁▇█▂▂█▁▁█
val_acc,▁▄▆▇▇████
val_loss_epoch,█▅▄▃▂▁▁▁▁
val_loss_step,█▇█▇▇▅▅▆▅▅▄▄▄▃▂▃▃▃▂▂▂▂▃▂▂▂▁▂▂▂▂▂▂▂▂▂▁▂▂▁
epoch,25
